# **Trading AI Agent**
In questo notebook andrò a mostrare come implementare bene un'agente AI che opera sulle stocks di Apple utilizzando Agentic AI. Agentic AI si riferisce a sistemi di intelligenza artificiale in grado di percepire il proprio ambiente, prendere decisioni e agire autonomamente per raggiungere un obiettivo. Questi agenti utilizzano tipicamente metodi di apprendimento per rinforzo (RL) per ottimizzare il loro comportamento nel tempo attraverso interazioni con l’ambiente.

Analizziamo i componenti principali dell'Agentic AI attraverso il nostro esempio:


*   **L'Agente**: L'agente è l'entità che prende decisioni all'interno del sistema AI. Nel nostro caso, l'agente di trading DQN sarà responsabile delle decisioni di trading basate sui dati di mercato.

*   **L'Ambiente:** L'ambiente è il sistema esterno in cui opera l'agente. Il nostro ambiente di trading sarà costituito dai dati del mercato azionario, in cui l'agente interagirà con le variazioni di prezzo ed eseguirà operazioni.
*   **Lo Stato:** Lo stato rappresenta le informazioni disponibili per l'agente in un dato momento. Lo stato del nostro agente di trading include il prezzo di chiusura delle azioni, le medie mobili e i rendimenti giornalieri.


*   **Lo Spazio delle Azioni:** Lo spazio delle azioni definisce le operazioni che l'agente può eseguire. Il nostro agente di trading ha tre possibili azioni: Acquistare, Vendere e Mantenere la posizione.

*   **La Funzione di Ricompensa:** La funzione di ricompensa valuta le prestazioni dell'agente assegnando un valore numerico alle sue azioni. L'obiettivo del nostro agente di trading sarà massimizzare il profitto totale entro la fine della sessione di trading







Per prima cosa, importerò tutte le librerie Python necessarie e raccoglierò i dati di mercato delle azioni Apple da Yahoo Finance:

In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque

# Definire il simbolo dell'azione e il periodo di tempo
symbol = "AAPL"
start_date = "2020-03-31"
end_date = "2025-03-31"

# download dati storici
data = yf.download(symbol, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


Ora calcolerò gli indicatori tecnici che aiuteranno l'agente AI a prendere decisioni di trading migliori:

In [12]:
# feature engineering
data['SMA_5'] = data['Close'].rolling(window=5).mean()
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['Returns'] = data['Close'].pct_change()

Ora rimuoviamo i valori mancanti e reimpostiamo l’index:

In [13]:
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

Successivamente, definiremo lo spazio delle azioni. L'agente AI ha tre possibili azioni:


*   **HOLD:** Non fare nulla.

*   **BUY:** Acquistare azioni.

*   **SELL:** Vendere le azioni detenute.






In [14]:
# definire lo spazio d'azione
AZIONI = {0: "HOLD", 1: "BUY", 2: "SELL"}

Ora estrarremo lo Stato dai dati:

In [15]:
# funzione get state
def get_state(data, index):
    return np.array([
        float(data.loc[index, 'Close']),
        float(data.loc[index, 'SMA_5']),
        float(data.loc[index, 'SMA_20']),
        float(data.loc[index, 'Returns'])
    ])

Questa funzione estrae la rappresentazione dello stato dal dataset in un dato indice temporale. Lo stato è un array che contiene:


*   **Closing price**

*   **5-day SMA**

*   **20-day SMA**
*  **Daily return percentage**

Questa rappresentazione numerica del mercato azionario viene fornita al modello AI per prendere decisioni di trading.




Ora definiremo un ambiente di trading per interagire con l'agente **AI Deep Q-Network (DQN)**, che gli permetterà di imparare a fare trading in modo redditizio:

In [16]:
# ambiente di trading
class TradingEnvironment:
    def __init__(self, data):
        self.data = data
        self.initial_balance = 10000
        self.balance = self.initial_balance
        self.holdings = 0
        self.index = 0

    def reset(self):
        self.balance = self.initial_balance
        self.holdings = 0
        self.index = 0
        return get_state(self.data, self.index)

    def step(self, action):
        price = float(self.data.loc[self.index, 'Close'])
        reward = 0

        if action == 1 and self.balance >= price:  # BUY
            self.holdings = self.balance // price
            self.balance -= self.holdings * price
        elif action == 2 and self.holdings > 0:  # SELL
            self.balance += self.holdings * price
            self.holdings = 0

        self.index += 1
        done = self.index >= len(self.data) - 1

        if done:
            reward = self.balance - self.initial_balance

        next_state = get_state(self.data, self.index) if not done else None
        return next_state, reward, done, {}

L'ambiente è implementato come una classe che simula il mercato azionario. Tieni traccia del bilancio dell'agente, delle sue partecipazioni e dell'indice di mercato corrente, e fornisce nuovi stati e ricompense in risposta alle azioni dell'agente.

# **Deep Q-Network (DQN)**
DQN è una rete neurale che approssima i valori Q per ogni coppia stato-azione. Ora definiremo l'architettura della rete neurale per il nostro Deep Q-Network. Sarà responsabile della previsione delle migliori azioni di trading basate sullo stato del mercato azionario:

In [17]:
# deep q-network
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

Qui abbiamo costruito un Deep Q-Network utilizzando PyTorch per ottimizzare le decisioni di trading azionario. Il modello presenta una rete neurale a tre strati per prevedere le azioni di trading, sfruttando l'attivazione ReLU per migliorare l'efficienza dell'apprendimento.

Esso produce i Q-values, che l'agente utilizza per determinare la miglior azione da intraprendere: **acquistare**, **vendere** o **mantenere**, in base alle condizioni di mercato.

Ora implementeremo l'agente AI che impara a fare trading azionario utilizzando il Deep Q-Learning. L'agente DQN interagirà con l'ambiente di trading, prenderà decisioni di trading (BUY,SELL,HOLD), memorizzerà le esperienze e imparerà da esse per migliorare le decisioni future:

In [18]:
# DQN agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = DQN(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.criterion = nn.MSELoss()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(list(AZIONI.keys()))
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            q_values = self.model(state)
        return torch.argmax(q_values).item()

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)

        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                next_state_tensor = torch.FloatTensor(next_state).unsqueeze(0)
                target += self.gamma * torch.max(self.model(next_state_tensor)).item()

            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            target_tensor = self.model(state_tensor).clone().detach()
            target_tensor[0][action] = target

            self.optimizer.zero_grad()
            output = self.model(state_tensor)
            loss = self.criterion(output, target_tensor)
            loss.backward()
            self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

Quindi, abbiamo sviluppato un agente Deep Q-Learning per interagire con l'ambiente del mercato azionario, migliorando le sue decisioni attraverso l'Experience Replay, che memorizza e riutilizza le esperienze passate per l'addestramento. L'agente bilancia efficacemente l'**Esplorazione vs. Sfruttamento**, prendendo inizialmente azioni casuali e facendo scelte più intelligenti man mano che l'apprendimento progredisce.

L'addestramento viene eseguito utilizzando batch di esperienze passate per affinare le prestazioni della rete neurale. Inoltre, viene applicato un fattore di sconto (gamma) per ponderare le ricompense immediate e future, al fine di garantire la redditività a lungo termine.

# **Training dell'Agente AI**
L'addestramento implica l'esecuzione di più episodi in cui l'agente interagisce con l'ambiente, impara dalle esperienze e aggiorna il suo modello. Ora alleniamo l'agente:

In [19]:
env = TradingEnvironment(data)
agent = DQNAgent(state_size=4, action_size=3)
batch_size = 32
episodes = 500
total_rewards = []

for episode in range(episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

    agent.replay(batch_size)
    total_rewards.append(total_reward)
    print(f"Episode {episode+1}/{episodes}, Ricompense ottenute: {total_reward}")

print("Training Completato!")

<ipython-input-15-69476ec516c5>:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'Close']),
<ipython-input-15-69476ec516c5>:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'SMA_5']),
<ipython-input-15-69476ec516c5>:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'SMA_20']),
<ipython-input-15-69476ec516c5>:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'Returns'])
<ipython-input-16-3a2d88ef8b5d>:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future

Episode 1/500, Ricompense ottenute: -9820.37873840332
Episode 2/500, Ricompense ottenute: 8093.963607788086
Episode 3/500, Ricompense ottenute: -9930.717819213867
Episode 4/500, Ricompense ottenute: -9826.102462768555
Episode 5/500, Ricompense ottenute: 10683.37777709961
Episode 6/500, Ricompense ottenute: -9845.937294006348
Episode 7/500, Ricompense ottenute: -9870.745506286621
Episode 8/500, Ricompense ottenute: -9881.17546081543
Episode 9/500, Ricompense ottenute: -9824.255729675293
Episode 10/500, Ricompense ottenute: -9995.802536010742
Episode 11/500, Ricompense ottenute: -9992.482437133789
Episode 12/500, Ricompense ottenute: -9822.856010437012
Episode 13/500, Ricompense ottenute: -9931.206802368164
Episode 14/500, Ricompense ottenute: -9864.580436706543
Episode 15/500, Ricompense ottenute: -9756.150779724121
Episode 16/500, Ricompense ottenute: -9865.678215026855
Episode 17/500, Ricompense ottenute: -9880.24114227295
Episode 18/500, Ricompense ottenute: 9846.268165588379
Episode

Qui, abbiamo addestrato l'Agente di Trading AI utilizzando Deep Q-Learning, simulando 500 sessioni di trading in cui l'agente ha imparato dall'esperienza. Ha sfruttato Esplorazione e Sfruttamento, inizialmente prendendo azioni casuali per poi fare decisioni più informate man mano che l'addestramento progrediva.

L'**Experience Replay** viene utilizzato per memorizzare le esperienze passate, permettendo alla rete neurale di apprendere tramite l'addestramento su batch. Durante tutto il processo, abbiamo monitorato le ricompense per misurare i miglioramenti delle prestazioni dell'agente nel tempo.

Dopo l'addestramento, possiamo testare l'agente su nuovi dati di mercato, consentendogli di prendere decisioni senza esplorazione casuale:

In [20]:
# Creo una nuova istanza dell'ambiente per il test.
test_env = TradingEnvironment(data)
state = test_env.reset()
done = False

# Simulo una sessione di trading utilizzando l'agente addestrato
while not done:
    # scegliere sempre l'azione migliore (exploitation)
    action = agent.act(state)
    next_state, reward, done, _ = test_env.step(action)
    state = next_state if next_state is not None else state

final_balance = test_env.balance
profit = final_balance - test_env.initial_balance
print(f"Bilancio finale dopo il testing: ${final_balance:.2f}")
print(f"Profitto Totale: ${profit:.2f}")

<ipython-input-15-69476ec516c5>:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'Close']),
<ipython-input-15-69476ec516c5>:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'SMA_5']),
<ipython-input-15-69476ec516c5>:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'SMA_20']),
<ipython-input-15-69476ec516c5>:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data.loc[index, 'Returns'])
<ipython-input-16-3a2d88ef8b5d>:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future

Bilancio finale dopo il testing: $26202.73
Profitto Totale: $16202.73


L'agente è partito con $10000 ed ha concluso con $26202.73, andando così in profitto di **$16202.73**

# **Sintesi:**
Quindi, ho  esplorato come costruire un agente di trading AI utilizzando Agentic AI e il Deep Q-Learning, permettendogli di prendere decisioni di trading autonome. Dopo l'addestramento, il nostro agente AI ha generato con successo un profitto positivo, dimostrando la sua capacità di affrontare le fluttuazioni del mercato.
